# Example usage

Here we will demonstrate how to use `SeisScan` to detect and locate earthquakes.

We will first read data from IRIS full wavefield experiment.

We will then proceed to compute characteristic function and backproject them for detection and location.